# Introduction

In this example we fit a standard GP and 'BasicRegressor' Heteroscedastic GP for the case where we have repeated observations for the same input.

# Initialise

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from HeteroscedasticGP.Models import BaseGP, BasicRegressor

# Make data

In [ ]:
# Fixing seed for example
np.random.seed(42)

# Inputs
n_repeat = 200
X = np.repeat(0, n_repeat)
X = np.append(X, np.repeat(1, n_repeat))
X = np.append(X, np.repeat(2, n_repeat))
X = np.vstack(X)
true_var = np.repeat(0.3, n_repeat)
true_var = np.append(true_var, np.repeat(1, n_repeat))
true_var = np.append(true_var, np.repeat(1.5, n_repeat))

# True function
f_true = np.sin(X).ravel()

# Generate noisy outputs
y = f_true + np.sqrt(true_var) * np.random.randn(len(X))

# True over new inputs
X_star = np.vstack(np.linspace(0, 2, 50))
f_star_true = np.sin(X_star).ravel()

# Extract true z
z_true = np.log(np.unique(true_var))


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, c="black", label="Observations")
plt.scatter(X, f_true, c="red", s=100, label="True function")
plt.legend()
plt.title("1D Regression with Heteroscedastic Noise")
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

# Pre-trained Model

Initialise models

In [ ]:
gp = BaseGP(ARD=False)
hgp = BasicRegressor(ARD=False)

Make initial guess at hyperparamters

In [ ]:
f_params0 = {'scale': 1.0, 'lengthscale': 1.0}
z_params0 = {'scale':1.0, 'lengthscale': 1.0}
z0 = np.zeros(3)
z0_mean = 0
noise_var0 = 1.

Assign initial guesses to models and make predictions

In [ ]:
gp.assign_hyperparameters(X, y, f_params=f_params0, noise_var=noise_var0)
hgp.assign_hyperparameters(X, y, f_params=f_params0, z_params=z_params0, z_opt=z0, z0_mean=z0_mean)

In [ ]:
mu_star_gp, var_star_gp = gp.predict(X_star)
mu_star_hgp, var_star_hgp, z_star = hgp.predict(X_star)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, c="black", label="Observations")
plt.plot(X_star, f_star_true, "black", lw=2, label="True function")

plt.plot(X_star, mu_star_gp, color='blue', label="Standard GP")
plt.fill_between(
    X_star.ravel(),
    mu_star_gp - 3 * np.sqrt(var_star_gp),
    mu_star_gp + 3 * np.sqrt(var_star_gp),
    color="blue", alpha=0.2
)

plt.plot(X_star, mu_star_hgp, color='red', label="Heteroscedastic GP")
plt.fill_between(
    X_star.ravel(),
    mu_star_hgp - 3 * np.sqrt(var_star_hgp),
    mu_star_hgp + 3 * np.sqrt(var_star_hgp),
    color="red", alpha=0.2
)
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

# Post-trained Model

Train models and print the best parameters

In [ ]:
gp.train(X, y, f_params0=f_params0, noise_var0=noise_var0)

In [ ]:
for keys in gp.f_params_opt.keys():
    print(f"f_param {keys}: {gp.f_params_opt[keys]}")
print("noise variance:", gp.noise_var)

In [ ]:
hgp.train(X, y, f_params0=f_params0, z_params0=z_params0, z0=z0, z0_mean=z0_mean)

In [ ]:
for keys in hgp.f_params_opt.keys():
    print(f"f_param {keys}: {hgp.f_params_opt[keys]}")
for keys in hgp.z_params_opt.keys():
    print(f"z_param {keys}: {hgp.z_params_opt[keys]}")

print('Most probable z:', hgp.z_opt)

Make predictions using the trained models

In [ ]:
mu_star_gp, var_star_gp = gp.predict(X_star)
mu_star_hgp, var_star_hgp, z_star = hgp.predict(X_star)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X, y, c="black", label="Observations")
plt.plot(X_star, f_star_true, "black", lw=2, label="True function")

plt.plot(X_star, mu_star_gp, color='blue', label="Standard GP")
plt.fill_between(
    X_star.ravel(),
    mu_star_gp - 3 * np.sqrt(var_star_gp),
    mu_star_gp + 3 * np.sqrt(var_star_gp),
    color="blue", alpha=0.2
)

plt.plot(X_star, mu_star_hgp, color='red', label="Heteroscedastic GP")
plt.fill_between(
    X_star.ravel(),
    mu_star_hgp - 3 * np.sqrt(var_star_hgp),
    mu_star_hgp + 3 * np.sqrt(var_star_hgp),
    color="red", alpha=0.2
)
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(hgp.Xu, hgp.z_opt, c="black", label="Inferred log-variances")
plt.plot(X_star, z_star, color="black")
plt.scatter(hgp.Xu, z_true, c="red", label="True log-variances")
plt.grid()
plt.legend()
plt.show()